## 예측 - song -> tag

In [44]:
df_song_train = pd.read_pickle('./raw_data/df_song_train.pkl')
df_song_val = pd.read_pickle('./raw_data/df_song_val.pkl')
val = pd.read_json('./raw_data/validation.json')

In [23]:
# 모델 불러오기
from keras.models import load_model
model = load_model('./model/multi_tag_classification_last_sgd.h5')

In [24]:
# y클래스 생성
from sklearn.preprocessing import MultiLabelBinarizer
raw_te_tags_mlb = MultiLabelBinarizer(sparse_output=True)
raw_te_tags = list(df_song_train.tags)
y = raw_te_tags_mlb.fit_transform(raw_te_tags)
y_classes = raw_te_tags_mlb.classes_

In [25]:
X = df_song_train[df_song_train.columns.difference(['tags'])]

In [26]:
raw_datas = pd.concat([X,df_song_val])
raw_datas.reset_index(inplace = True)
raw_datas.drop_duplicates('id', inplace = True,keep = 'last')
raw_datas.set_index('id', inplace = True)

,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,GN0203,...,issue_year_2015,issue_year_2016,issue_year_2017,issue_year_2018,issue_year_2019,issue_year_2020,issue_year_2022,like_cnt,population,tag_score
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000089,0.000460,0.065164
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000119,0.000460,0.009413
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001196,0.001840,0.159218
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000073,0.000000,0.013426
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000009,0.000000,0.005898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
707962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
707965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008497,0.015639,0.372112


In [28]:
raw_datas = raw_datas[raw_datas.columns.difference(['tags','population','tag_score','like_cnt'])]

In [45]:
from tqdm.notebook import tqdm
val['predict_tags'] = 0
for i in tqdm(range(0,len(val['songs']))):
    k = int(val['songs'][i][0]) # 중복제거 고민 코드 지금은 앞에 1개의 태그에서만 가져옴
    val_data = raw_datas.loc[k].values
    preds = model.predict(val_data.reshape(1,-1))
    val['predict_tags'][i] = list(y_classes[np.concatenate(preds.argsort())[::-1][1:11]])

In [46]:
# 혹시나 태그 10개 안채워진거있나 확인
val[val['predict_tags'].apply(lambda x : len(x) != 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date,predict_tags


In [47]:
val

,tags,id,plylst_title,songs,like_cnt,updt_date,predict_tags
0,[OST],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000,"[감성, 힐링, 기분전환, 잔잔한, 휴식, 팝, 새벽, 카페, 밤, 매장음악]"
1,[턴테이블],131447,앨리스테이블,"[572238, 233146, 61064, 336013, 611334, 359175...",1,2014-07-16 15:24:24.000,"[휴식, 힐링, 감성, 드라이브, 잔잔한, 새벽, 팝, 사랑, 밤, 카페]"
2,[발라드],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000,"[기분전환, 힐링, 드라이브, 감성, 잔잔한, 새벽, 밤, 추억, 팝, 카페]"
3,[발라드],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000,"[휴식, 드라이브, 감성, 힐링, 잔잔한, 새벽, 밤, 팝, 사랑, 발라드]"
4,[CCM],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000,"[감성, 새벽, 밤, 휴식, 팝, 잔잔한, 힐링, 카페, 기분전환, 여행]"
...,...,...,...,...,...,...,...
23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000,"[감성, 휴식, 드라이브, 힐링, 잔잔한, 밤, 새벽, 사랑, 팝, 카페]"
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000,"[드라이브, 휴식, 잔잔한, 힐링, 감성, 새벽, 밤, 팝, 사랑, 카페]"
23012,[팝],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000,"[드라이브, 휴식, 감성, 잔잔한, 힐링, 팝, 밤, 새벽, 카페, 사랑]"
23013,[클래식],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000,"[휴식, 감성, 드라이브, 힐링, 잔잔한, 팝, 새벽, 사랑, 카페, 밤]"


## 예측 - tag -> song

In [6]:
df_tags_train = pd.read_pickle('./raw_data/df_tags_train.pkl')
df_tags_val = pd.read_pickle('./raw_data/df_tags_val.pkl')
# val = pd.read_json('datas/validation.json')

In [7]:
# 모델 불러오기
from keras.models import load_model
model = load_model('./model/multi_song_classification_last.h5')

Using TensorFlow backend.


In [8]:
# y클래스 생성
from sklearn.preprocessing import MultiLabelBinarizer
raw_te_tags_mlb = MultiLabelBinarizer(sparse_output=True)
raw_te_tags = list(df_tags_train.songs)
y = raw_te_tags_mlb.fit_transform(raw_te_tags)
y_classes = raw_te_tags_mlb.classes_

In [9]:
X = df_tags_train[df_tags_train.columns.difference(['songs'])]

In [10]:
raw_datas = pd.concat([X,df_tags_val])
raw_datas.reset_index(inplace = True)
raw_datas.drop_duplicates('tags', inplace = True,keep = 'last')
raw_datas.set_index('tags', inplace = True)
raw_datas

,GN0100,GN0101,GN0102,GN0103,GN0104,GN0105,GN0200,GN0201,GN0202,GN0203,...,GN2802,GN2803,GN2804,GN2805,GN2806,GN2900,GN2901,GN2902,GN2903,GN3000
tags,,,,,,,,,,,,,,,,,,,,,
,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
00,0.000006,0.000006,0.000000,0.000000,0.000000,0.000000,0.000374,0.000360,0.000000,0.000965,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
007시리즈,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
00s,0.000360,0.000345,0.001471,0.002077,0.000702,0.000029,0.000505,0.000483,0.000000,0.002342,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
00년,0.000262,0.000259,0.000000,0.000383,0.000539,0.000092,0.000677,0.000648,0.002062,0.000413,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
힙합엘이,0.000009,0.000009,0.000000,0.000000,0.000000,0.000011,0.000333,0.000319,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.006789,0.006821,0.0,0.012837,0.0
힙합재즈,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
힙합존,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0


In [11]:
# 중복제거 고민
def find_songs(x):
    return list(OrderedDict.fromkeys(np.concatenate([y_classes[np.concatenate(model.predict(raw_datas.loc[i].values.reshape(1,-1)).argsort())[::-1][1:int(201/len(x))]] for i in x])))[:100]

In [12]:
from tqdm.notebook import tqdm
from collections import OrderedDict
tqdm.pandas()
val['predict_songs'] = val['tags'].progress_apply(lambda x: find_songs(x))

In [14]:
# 혹시나 노래 100개 안채워진거있나 확인
not_100 = val[val['predict_songs'].apply(lambda x : len(x) != 100)]
len(not_100['predict_songs']) 

4459

In [18]:
song_tags = pd.read_json('./raw_data/song_tags.json', typ = 'frame')
import random
random.seed(13)
song_list = list(song_tags['songs'])
for i in not_100.index:
        s = list(map(int,random.sample(song_list, 100 - len(not_100['predict_songs'][i]))))
        not_100['predict_songs'][i] = list(set(s + not_100['predict_songs'][i]))

In [19]:
not_100[not_100['predict_songs'].apply(lambda x : len(x) != 100)]

,tags,id,plylst_title,songs,like_cnt,updt_date,predict_songs


In [49]:
val = pd.concat([val,not_100])
val = val.drop_duplicates('id',keep = 'last')
val.reset_index(drop = True, inplace = True)

In [50]:
# 노래 중복확인
no_song = []
for i in val.index:
    if len(set(val['predict_songs'][i])) != 100:
        no_song.append(val.loc[i])

TypeError: 'float' object is not iterable

In [51]:
# 노래 중복확인
no_tag = []
for i in val.index:
    if len(set(val['predict_tags'][i])) != 10:
        no_tag.append(val.loc[i])

TypeError: 'float' object is not iterable

In [45]:
no_song

[]

In [44]:
no_tag

[]

#### 제출용 저장

In [53]:
result = val[['id','predict_songs','predict_tags']]

In [54]:
results = []
for i in range(0,len(result)):
    results.append({'id' : int(result['id'][i]), 'songs' : list(map(int, result['predict_songs'][i])), 'tags' : result['predict_tags'][i]})

TypeError: 'float' object is not iterable

In [55]:
import json
with open('results.json', 'w', encoding='utf-8',) as make_file:
    json.dump(results, make_file, indent="\t", ensure_ascii=False)